# BEHAVIOR Prep 3 — Transforming Meshes with SE(3) (Real Bug Patterns)
**Aim:** Correctly apply SE(3) transforms to meshes and recognize common production mistakes.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
np.set_printoptions(precision=4, suppress=True)


In [ ]:
def skew(w):
    wx, wy, wz = w
    return np.array([[0, -wz, wy],
                     [wz, 0, -wx],
                     [-wy, wx, 0]], float)

def make_T(R, p):
    T = np.eye(4)
    T[:3,:3] = R
    T[:3,3] = p
    return T

def inv_T(T):
    R = T[:3,:3]
    p = T[:3,3]
    Rt = R.T
    return make_T(Rt, -Rt @ p)

def rotz(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c,-s,0],[s,c,0],[0,0,1]], float)

def roty(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c,0,s],[0,1,0],[-s,0,c]], float)

def plot_frame(ax, T, name="", L=0.12):
    o = T[:3,3]
    R = T[:3,:3]
    cols = ["r","g","b"]
    for i in range(3):
        ax.plot([o[0], o[0]+R[0,i]*L],
                [o[1], o[1]+R[1,i]*L],
                [o[2], o[2]+R[2,i]*L], c=cols[i], linewidth=2)
    if name:
        ax.text(o[0], o[1], o[2], " "+name)

def set_axes_equal(ax, lim=1.0):
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.set_zlim(-lim, lim)
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")


## 1) Correct: homogeneous coordinates


In [ ]:
import numpy as np

V_obj = np.random.randn(500,3)*0.05
T_wo = make_T(rotz(np.deg2rad(30)) @ roty(np.deg2rad(15)), np.array([0.4, -0.1, 0.2]))

V_h = np.c_[V_obj, np.ones(len(V_obj))]
V_world = (T_wo @ V_h.T).T[:, :3]

print("First vertex obj:", V_obj[0])
print("First vertex world:", V_world[0])


## 2) Bug A: rotation but no translation


In [ ]:
R = T_wo[:3,:3]
V_world_bugA = (R @ V_obj.T).T
print("Mean world (correct):", V_world.mean(axis=0))
print("Mean world (bugA):   ", V_world_bugA.mean(axis=0))


## 3) Bug B: row/column convention mismatch


In [ ]:
V_world_bugB = V_obj @ R  # wrong under column-vector convention
V_world_row_ok = V_obj @ R.T + T_wo[:3,3]

print("||bugB - correct|| (first 5):", np.linalg.norm(V_world_bugB[:5] - V_world[:5]))
print("||row_ok - correct|| (first 5):", np.linalg.norm(V_world_row_ok[:5] - V_world[:5]))


## 4) Visualize correct vs bugA


In [ ]:
fig = plt.figure(figsize=(7,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(V_world[:,0], V_world[:,1], V_world[:,2], s=3, label="correct")
ax.scatter(V_world_bugA[:,0], V_world_bugA[:,1], V_world_bugA[:,2], s=3, label="bugA (no translation)")

plot_frame(ax, np.eye(4), "w", L=0.2)
plot_frame(ax, T_wo, "o in w", L=0.2)
set_axes_equal(ax, lim=1.0)
plt.title("Mesh transform: correct vs common bug")
plt.legend()
plt.show()
